In [1]:
# Import the important packages
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
# Load the raw data
gamesdf = pd.read_csv('../../data/games.csv')

In [3]:
gamesdf.columns

Index(['Unnamed: 0', 'away.teamStats.teamSkaterStats.blocked',
       'away.teamStats.teamSkaterStats.faceOffWinPercentage',
       'away.teamStats.teamSkaterStats.giveaways',
       'away.teamStats.teamSkaterStats.goals',
       'away.teamStats.teamSkaterStats.hits',
       'away.teamStats.teamSkaterStats.pim',
       'away.teamStats.teamSkaterStats.powerPlayGoals',
       'away.teamStats.teamSkaterStats.powerPlayOpportunities',
       'away.teamStats.teamSkaterStats.powerPlayPercentage',
       'away.teamStats.teamSkaterStats.shots',
       'away.teamStats.teamSkaterStats.takeaways', 'content.link', 'date',
       'gameDate', 'gameID', 'gamePk', 'gameType',
       'home.teamStats.teamSkaterStats.blocked',
       'home.teamStats.teamSkaterStats.faceOffWinPercentage',
       'home.teamStats.teamSkaterStats.giveaways',
       'home.teamStats.teamSkaterStats.goals',
       'home.teamStats.teamSkaterStats.hits',
       'home.teamStats.teamSkaterStats.pim',
       'home.teamStats.teamSkate

In [4]:
def create_lag(df, p, sortList, partList, lagList):
    """
    Cette fonction prend comme intrant un df, une liste de variables pour lesquelles il faut
    creer des valeurs retardees, le nombre de retard, les variables sur lesquelles il faut sort,
    les variables de partition et la liste de varibles pour lesquelles on veut des valeurs retardees.
    La fonction retourne un df augmente des variables retardees
    """
    df = df.sort_values(by=sortList)
    
    for feature in lagList:
        for i in range(p):
            df['{}.L{}'.format(feature,i+1)] = df.groupby(partList)[feature].shift(i+1)
    
    return df

In [5]:
# List des features

features5L = ['away.teamStats.teamSkaterStats.blocked','away.teamStats.teamSkaterStats.faceOffWinPercentage',
              'away.teamStats.teamSkaterStats.giveaways','away.teamStats.teamSkaterStats.goals',
              'away.teamStats.teamSkaterStats.hits','away.teamStats.teamSkaterStats.pim',
              'away.teamStats.teamSkaterStats.powerPlayGoals','away.teamStats.teamSkaterStats.powerPlayOpportunities',
              'away.teamStats.teamSkaterStats.powerPlayPercentage','away.teamStats.teamSkaterStats.shots',
              'away.teamStats.teamSkaterStats.takeaways','home.teamStats.teamSkaterStats.blocked',
              'home.teamStats.teamSkaterStats.faceOffWinPercentage','home.teamStats.teamSkaterStats.giveaways',
              'home.teamStats.teamSkaterStats.goals','home.teamStats.teamSkaterStats.hits',
              'home.teamStats.teamSkaterStats.pim','home.teamStats.teamSkaterStats.powerPlayGoals',
              'home.teamStats.teamSkaterStats.powerPlayOpportunities','home.teamStats.teamSkaterStats.powerPlayPercentage',
              'home.teamStats.teamSkaterStats.shots','home.teamStats.teamSkaterStats.takeaways']

features1L = ['teams.away.leagueRecord.losses','teams.away.leagueRecord.ot',
              'teams.away.leagueRecord.wins','teams.home.leagueRecord.losses',
              'teams.home.leagueRecord.ot','teams.home.leagueRecord.wins']

#Variables date, saisons et equipes
state_time = ['teams.home.team.name', 'season', 'gamePk']


In [6]:
# Creation de la variable a predire -- home win
gamesdf['game.homewin'] = (gamesdf['teams.home.score'] - gamesdf['teams.away.score']> 0).astype('int')

In [7]:
gamesdf_ = gamesdf[['game.homewin','teams.away.team.name'] + features5L + features1L + state_time]

In [8]:
gamesdf_ = create_lag(df=gamesdf_,
                      p=1,
                      sortList=['teams.home.team.name', 'season', 'gamePk'],
                      partList=['teams.home.team.name', 'season'],
                      lagList=features1L)

In [9]:
gamesdf_ = create_lag(df=gamesdf_,
                      p=10,
                      sortList=['teams.home.team.name', 'season', 'gamePk'],
                      partList=['teams.home.team.name', 'season'],
                      lagList=features5L)

In [10]:
# Maintenant, on drop les variables qu'on utilisera pas pour predire....
gamesdf_final = gamesdf_.drop(columns=['gamePk'] + features1L + features5L)

In [11]:
x = gamesdf_final['teams.away.team.name'].value_counts()>50
x = x[x].index.values
list(x)

['Pittsburgh Penguins',
 'Chicago Blackhawks',
 'New York Rangers',
 'Washington Capitals',
 'Detroit Red Wings',
 'Boston Bruins',
 'San Jose Sharks',
 'Philadelphia Flyers',
 'Nashville Predators',
 'Anaheim Ducks',
 'Los Angeles Kings',
 'Tampa Bay Lightning',
 'St. Louis Blues',
 'Montréal Canadiens',
 'Ottawa Senators',
 'Minnesota Wild',
 'Vancouver Canucks',
 'New Jersey Devils',
 'Dallas Stars',
 'Columbus Blue Jackets',
 'Toronto Maple Leafs',
 'New York Islanders',
 'Calgary Flames',
 'Florida Panthers',
 'Colorado Avalanche',
 'Buffalo Sabres',
 'Carolina Hurricanes',
 'Edmonton Oilers',
 'Winnipeg Jets',
 'Phoenix Coyotes',
 'Arizona Coyotes',
 'Atlanta Thrashers',
 'Vegas Golden Knights']

In [17]:
gamesdf_final_ = gamesdf_final.loc[gamesdf_final['teams.away.team.name'].isin(x) &
                                   gamesdf_final['teams.home.team.name'].isin(x)]

In [18]:
gamesdf_final_.describe()

,game.homewin,season,teams.away.leagueRecord.losses.L1,teams.away.leagueRecord.ot.L1,teams.away.leagueRecord.wins.L1,teams.home.leagueRecord.losses.L1,teams.home.leagueRecord.ot.L1,teams.home.leagueRecord.wins.L1,away.teamStats.teamSkaterStats.blocked.L1,away.teamStats.teamSkaterStats.blocked.L2,away.teamStats.teamSkaterStats.blocked.L3,away.teamStats.teamSkaterStats.blocked.L4,away.teamStats.teamSkaterStats.blocked.L5,away.teamStats.teamSkaterStats.blocked.L6,away.teamStats.teamSkaterStats.blocked.L7,away.teamStats.teamSkaterStats.blocked.L8,away.teamStats.teamSkaterStats.blocked.L9,away.teamStats.teamSkaterStats.blocked.L10,away.teamStats.teamSkaterStats.faceOffWinPercentage.L1,away.teamStats.teamSkaterStats.faceOffWinPercentage.L2,away.teamStats.teamSkaterStats.faceOffWinPercentage.L3,away.teamStats.teamSkaterStats.faceOffWinPercentage.L4,away.teamStats.teamSkaterStats.faceOffWinPercentage.L5,away.teamStats.teamSkaterStats.faceOffWinPercentage.L6,away.teamStats.teamSkaterStats.faceOffWinPercentage.L7,away.teamStats.teamSkaterStats.faceOffWinPercentage.L8,away.teamStats.teamSkaterStats.faceOffWinPercentage.L9,away.teamStats.teamSkaterStats.faceOffWinPercentage.L10,away.teamStats.teamSkaterStats.giveaways.L1,away.teamStats.teamSkaterStats.giveaways.L2,away.teamStats.teamSkaterStats.giveaways.L3,away.teamStats.teamSkaterStats.giveaways.L4,away.teamStats.teamSkaterStats.giveaways.L5,away.teamStats.teamSkaterStats.giveaways.L6,away.teamStats.teamSkaterStats.giveaways.L7,away.teamStats.teamSkaterStats.giveaways.L8,away.teamStats.teamSkaterStats.giveaways.L9,away.teamStats.teamSkaterStats.giveaways.L10,away.teamStats.teamSkaterStats.goals.L1,away.teamStats.teamSkaterStats.goals.L2,away.teamStats.teamSkaterStats.goals.L3,away.teamStats.teamSkaterStats.goals.L4,away.teamStats.teamSkaterStats.goals.L5,away.teamStats.teamSkaterStats.goals.L6,away.teamStats.teamSkaterStats.goals.L7,away.teamStats.teamSkaterStats.goals.L8,away.teamStats.teamSkaterStats.goals.L9,away.teamStats.teamSkaterStats.goals.L10,away.teamStats.teamSkaterStats.hits.L1,away.teamStats.teamSkaterStats.hits.L2,away.teamStats.teamSkaterStats.hits.L3,away.teamStats.teamSkaterStats.hits.L4,away.teamStats.teamSkaterStats.hits.L5,away.teamStats.teamSkaterStats.hits.L6,away.teamStats.teamSkaterStats.hits.L7,away.teamStats.teamSkaterStats.hits.L8,away.teamStats.teamSkaterStats.hits.L9,away.teamStats.teamSkaterStats.hits.L10,away.teamStats.teamSkaterStats.pim.L1,away.teamStats.teamSkaterStats.pim.L2,away.teamStats.teamSkaterStats.pim.L3,away.teamStats.teamSkaterStats.pim.L4,away.teamStats.teamSkaterStats.pim.L5,away.teamStats.teamSkaterStats.pim.L6,away.teamStats.teamSkaterStats.pim.L7,away.teamStats.teamSkaterStats.pim.L8,away.teamStats.teamSkaterStats.pim.L9,away.teamStats.teamSkaterStats.pim.L10,away.teamStats.teamSkaterStats.powerPlayGoals.L1,away.teamStats.teamSkaterStats.powerPlayGoals.L2,away.teamStats.teamSkaterStats.powerPlayGoals.L3,away.teamStats.teamSkaterStats.powerPlayGoals.L4,away.teamStats.teamSkaterStats.powerPlayGoals.L5,away.teamStats.teamSkaterStats.powerPlayGoals.L6,away.teamStats.teamSkaterStats.powerPlayGoals.L7,away.teamStats.teamSkaterStats.powerPlayGoals.L8,away.teamStats.teamSkaterStats.powerPlayGoals.L9,away.teamStats.teamSkaterStats.powerPlayGoals.L10,away.teamStats.teamSkaterStats.powerPlayOpportunities.L1,away.teamStats.teamSkaterStats.powerPlayOpportunities.L2,away.teamStats.teamSkaterStats.powerPlayOpportunities.L3,away.teamStats.teamSkaterStats.powerPlayOpportunities.L4,away.teamStats.teamSkaterStats.powerPlayOpportunities.L5,away.teamStats.teamSkaterStats.powerPlayOpportunities.L6,away.teamStats.teamSkaterStats.powerPlayOpportunities.L7,away.teamStats.teamSkaterStats.powerPlayOpportunities.L8,away.teamStats.teamSkaterStats.powerPlayOpportunities.L9,away.teamStats.teamSkaterStats.powerPlayOpportunities.L10,away.teamStats.teamSkaterStats.powerPlayPercentage.L1,away.teamStats.teamSkaterStats.powerPlayPercentage.L2,away.teamStats.teamSkaterStats.

In [19]:
# team away
gamesdf_final_['teams.away.leagueRecord.gameplayed.L1'] = gamesdf_final_['teams.away.leagueRecord.losses.L1'] + gamesdf_final_['teams.away.leagueRecord.wins.L1'] + gamesdf_final_['teams.away.leagueRecord.ot.L1']
# team home
gamesdf_final_['teams.home.leagueRecord.gameplayed.L1'] = gamesdf_final_['teams.home.leagueRecord.losses.L1'] + gamesdf_final_['teams.home.leagueRecord.wins.L1'] + gamesdf_final_['teams.home.leagueRecord.ot.L1']



# Nombre de points = ot + wins * 2

# team away
gamesdf_final_['teams.away.leagueRecord.points.L1'] = ( gamesdf_final_['teams.away.leagueRecord.wins.L1'] * 2 ) + gamesdf_final_['teams.away.leagueRecord.ot.L1']
# team home
gamesdf_final_['teams.home.leagueRecord.points.L1'] = ( gamesdf_final_['teams.home.leagueRecord.wins.L1'] * 2 ) + gamesdf_final_['teams.home.leagueRecord.ot.L1']



# Points Per Game = points / gp

# team away
gamesdf_final_['teams.away.leagueRecord.ppg.L1'] = gamesdf_final_['teams.away.leagueRecord.points.L1'] / gamesdf_final_['teams.away.leagueRecord.gameplayed.L1']
# team home
gamesdf_final_['teams.home.leagueRecord.ppg.L1'] = gamesdf_final_['teams.home.leagueRecord.points.L1'] / gamesdf_final_['teams.home.leagueRecord.gameplayed.L1']



# Difference PPG entre Home vs Away = home PPG - away PPG

gamesdf_final_['game.ppgdiff_home.L1'] = gamesdf_final_['teams.home.leagueRecord.ppg.L1'] - gamesdf_final_['teams.away.leagueRecord.ppg.L1']

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [20]:
# Export en csv..
gamesdf_final.to_csv('../../data/model_input_v2.csv')